# BUILDING FAST QUERIES ON A CSV

In this project am going to use the laptops.csv from the kaggle.

I am going to use the dataset to answer a few business questions.

# Dataset

Importing the dataset
Am using the UTF-8 encoding on the dataset.

Opening the file pandas inform of a table

In [1]:
import pandas as pd
ltp = pd.read_csv('laptops.csv', encoding = 'UTF-8')

Store the header row in header and the rest of the rows in rows

In [2]:
header = ltp[:0]
header

,Id,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price


print the first 5 rows in rows from the dataset.

In [3]:
rows = ltp[1:]
rows[:5]

,Id,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
1,7287764,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898
2,3362737,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575
3,9722156,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537
4,8550527,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803
5,8529462,Acer,Aspire 3,Notebook,15.6,1366x768,AMD A9-Series 9420 3GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.1kg,400


Opening the dataset with csv inform of list of list

In [3]:
with open('laptops.csv') as t:
    reader = csv.reader(t)
    file_t = list(reader)

# Implementing a class constructor.

The goal of this guided project is to **create a class that represents our inventory**. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Here are some queries that we will want to answer:

Given a laptop id, find the corresponding data.

Given an amount of money, find whether there are two laptops whose total price is that given amount.

Identify all laptops whose price falls within a given budget.

In [2]:
import csv
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
inventory = Inventory('laptops.csv')
print(inventory.header)
print(inventory.rows[:3])
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', 898], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]]
1303


In [4]:
print(file_t[0])

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']


# Finding a laptop using an ID

In [5]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
            
    def get_laptop_from_id(self,laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
        return None


In [6]:
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id('3362737'))

print(inventory.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


# Improving ID lookup

Here I am improving the Time Complexity algorithm. Am doing this by improving it through pre-processing the dataset using dictionaries to be able to return the laptop id row if the laptop id is present and None if its not in the datset.

In [7]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self,laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None 

In [8]:
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id_fast('3362737'))
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


In [9]:
import random
import time

inventory = Inventory('laptops.csv')

total_time_no_dict= 0
ids = [str(random.randint(1000000,9999999)) for _ in range(10000)]

for id in ids:
    start = time.time()
    inventory.get_laptop_from_id(id)
    end = time.time()
    runtime = end -start
    total_time_no_dict+= runtime

total_time_dict = 0
for id in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(id)
    end = time.time()
    runtime = end -start
    total_time_dict+= runtime  
    
    


In [10]:
dic = (total_time_dict)
no_dic = (total_time_no_dict)
results = no_dic/dic
print(results)

222.24425770308125


Using the dictionary preprocessor improves the model by 227 times fast than before preprocessing


# Two laptop Promotion

In [23]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self,laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None 
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if dollars == row[-1]:
                return True
        for x in self.rows:
            for y in self.rows:
                if x[-1] + y[-1] == dollars:
                    return True
        return False
    

In [24]:
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))

True
False


# Optimizing laptop Promotion

In [36]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
            
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        self.prices= set()
        for row in self.rows:
            self.prices.add(row[-1])
            
    def get_laptop_from_id(self,laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None 
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if dollars == row[-1]:
                return True
        for x in self.rows:
            for y in self.rows:
                if x[-1] + y[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for x in self.prices:
            if dollars - x in self.prices:
                return True
        return False

In [37]:
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars_fast(1000))
print(inventory.check_promotion_dollars_fast(442))

True
False


In [27]:
inventory = Inventory('laptops.csv')

total_time_no_set = 0
prices = [random.randint(100,5000) for _ in range(100)]

for price in prices:
    start = time.time()
    inventory.check_promotion_dollars(price)
    end = time.time()
    runtime = end -start
    total_time_no_set += runtime

total_time_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(price)
    end = time.time()
    runtime = end -start
    total_time_set += runtime  

In [28]:
print(total_time_set)
print(total_time_no_set)

results = 2.217395067214966/0.0009388923645019531

print(results)

0.0004429817199707031
1.0984554290771484
2361.713814118842


After preprocessing using a set we got 0.0009388923645019531 and before processing 2.217395067214966.

This was 2361 times faster after preprocessing! 
I love the magic😊

# Finding a laptop within a budget

In [54]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
            
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        self.prices= set()
        for row in self.rows:
            self.prices.add(row[-1])
            
        self.rows_by_price = sorted(self.rows, key=row_price)
            
    def get_laptop_from_id(self,laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None 
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if dollars == row[-1]:
                return True
        for x in self.rows:
            for y in self.rows:
                if x[-1] + y[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for x in self.prices:
            if dollars - x in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self,target_price):
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        while range_start < range_end:
            mid_value = (range_start + range_end)//2
            price = self.rows_by_price[mid_value][-1]
            if price == target_price:
                return mid_value
            elif price < target_price:
                range_start = mid_value + 1
            else:
                range_end = mid_value - 1
        price = self.rows_by_price[range_start][-1]
        if price != target_price:
            return -1
        return range_start

In [57]:
inventory = Inventory('laptops.csv')
print(inventory.find_first_laptop_more_expensive(1000))

print(inventory.find_first_laptop_more_expensive(10000))

682
-1


In [ ]:
That's the function to find the first laptop that's above the target price. All laptops below that